## Preprocessing the data from the Market Research Survey

In [ ]:
# import libraries
import pandas as pd

# import csv file as dataframe
data_path = "tudublin_amenities_access_survey.csv"
survey_data = pd.read_csv(data_path, delimiter=",", encoding='unicode_escape')
survey_data.head(5)

In [ ]:
# check number of rows and columns
print("Number of columns: ", len(survey_data.columns))
print("Number of rows: ", len(survey_data))

In [ ]:
# check data type of columns -- all object
print("Data types of columns:")
survey_data.dtypes

### First order of business: clean up the df
<li> I will start by removing the unecessary columns: start & completion time, name & email because they're empty.<br></li>
<li>Need to rename the columns as well</li>
<li>Next, I will fill in the NAN in county responses, and group the different "other" responses in job<br></li>
<li>Then, I will split the df into those who are employed and unemployed, and then those who use public amenities or not in their professional life.</li>

In [ ]:
# remove unecessary columns
survey_data = survey_data.drop(survey_data.columns[[1,2,3,4]], axis=1)

# import column names
col_file = open("col_names.txt", "r")
col_names = col_file.read()
col_list = col_names.replace(' ','').split(",")
col_file.close()

# rename columns
survey_data = survey_data.set_axis(col_list, axis=1)
print(survey_data.columns)

In [ ]:
# remove trailing semicolons in all columns
survey_data = survey_data.map(lambda x: x.rstrip(';') if isinstance(x, str) else x)

In [ ]:
# group the "other" answers together in new/existing categories using dictionaries and mapping
sector_list_replace = ["Education","Government", "Construction", "Hospitality", "Technology", "Other"]

sector_dict = {
    sector_list_replace[0] : ["Student"],
    sector_list_replace[1] : ["Local Government","COUNTY COUNCIL"],
    sector_list_replace[2] : ["Architecture"],
    sector_list_replace[3] : ["Services (events)","Food","Fast food","Food Service","restaurant"],
    sector_list_replace[4] : ["IT","Software Development","Technology/Finance","Data","IT solutions and services.","Cybersecurity ","Tech"],
    sector_list_replace[5] : ["Pharmaceuticals","Arts","Mechanic"]
}

sector_map = {item: sector for sector, items in sector_dict.items() for item in items}
survey_data["sector"] = survey_data["sector"].map(sector_map).fillna(survey_data["sector"])

<li>I combined food services, restaurant, fast food and service events under hospitality
<li>I combined the different versions of tech, cybersecurity, IT under techonology
<li>I put architecture under construction
<li>I put student under education
<li>Any other sector which constituted 1 response, I put it under Other


In [ ]:
# split in 2 sub dataframe, those that use amenity data for work and those who dont
# user A = dont' use amenity data
# user B = use amenity data

users_A = pd.DataFrame(survey_data[survey_data["use_amenity_data"] == "No"])
users_B = pd.DataFrame(survey_data[survey_data["use_amenity_data"] == "Yes"])

#### Pre process user_A = Those that DONT USE amenity data

In [ ]:
# remove useless columns aka those with only NAN values
users_A = users_A.dropna(axis=1, how='all')


# replace NaN with "unapplicable" in columns users did not answer (branching) and "empty" with those users chose to not answer
## made a mistake in the branching for this section, other_amenity sends to contact, demo sends to other feature instead of other amenity as well

A_branch_cols_list = users_A.columns[7:9].tolist()
A_unrequired_cols_list = users_A.columns[9:].tolist()

def solve_nan(df, col_list, value):
    for col in col_list:
        df[col] = df[col].fillna(value)

solve_nan(users_A, A_branch_cols_list, "Unapplicable")
solve_nan(users_A, A_unrequired_cols_list, "Empty")

In [ ]:
# function to handle "other" answers
def prefix_other_answers(row, answer):
    items = row.split(";") # split
    updated_items = [
        item if item.strip() in answer else f"Other: {item.strip()}"
        for item in items
    ]
    return "; ".join(updated_items)  # join back

# adding "other" prefix to why impractical demo personal answers
neg_reason_demo_list = ["Already have access to this information","I don't like web applications","Empty","Unapplicable"]
users_A["why_impractical_demo_personal"] = users_A["why_impractical_demo_personal"].apply(prefix_other_answers, answer=neg_reason_demo_list)

# adding "other" prefix to additional amenity "other" answers
other_amenity_list = ["Bike lanes","Bike sheds","Hiking trails","Car parking","Parks","Public bathrooms","Empty","Unapplicable"]
users_B["other_amenity_work"] = users_B["other_amenity_work"].apply(prefix_other_answers, answer=other_amenity_list)

In [ ]:
users_A.columns

In [ ]:
# explode multiple answers cols 

def explode_multiple_answers(data, column_names, delimiter=';'):
    """
    Function to plot the count of devices in a specified column, handling multiple answers.
    
    Parameters:
    - data: DataFrame containing the survey data
    - column_name: The name of the column to analyze (e.g., 'device_personal')
    - delimiter: The delimiter separating multiple values (default is ';')
    
    Returns:
    - A Plotly bar chart
    """
    # Loop through the list of columns and apply the split and explode process
    for column_name in column_names:
        if column_name in data.columns:
            # Check if the delimiter exists in any of the rows of the column
            if data[column_name].str.contains(delimiter).any():
                # Split the values in the column by the delimiter
                data[column_name + '_exploded'] = data[column_name].str.split(delimiter)
                # Explode the column to create multiple rows
                data = data.explode(column_name + '_exploded')
            else:
                # If no delimiter is found, no need to split or explode
                data[column_name + '_exploded'] = data[column_name]  # Keep original values in a new column
    
    return data

userAcols_to_explode = ["device_personal","why_impractical_demo_personal","other_amenity_personal","other_feature_personal"]

users_A_expl = explode_multiple_answers(users_A,userAcols_to_explode)

In [ ]:
users_A_expl.head(10)

#### Export new csv user_A

In [ ]:
users_A_expl.to_csv("userA_responses.csv", index=False)

#### Preprocess user_B = Those that USE amenity data

In [ ]:
# replace NAN in first row county with Mayo (Damian)
users_B["county"] = users_B["county"].fillna("Mayo")

# remove useless columns aka those with only NAN values
users_B = users_B.dropna(axis=1, how='all')

# replace NaN with "unapplicable" in columns users did not answer (branching) and "empty" with those users chose to not answer

branch_cols_list = ["why_unsatisfied_tool_work", "why_impractical_demo_work"]
unrequired_cols_list = users_B.columns[[11]].tolist() + users_B.columns[15:].tolist()

def solve_nan(df, col_list, value):
    for col in col_list:
        df[col] = df[col].fillna(value)

solve_nan(users_B, branch_cols_list, "Unapplicable")
solve_nan(users_B, unrequired_cols_list, "Empty")

In [ ]:
# shorten answers for type amenity and type tool
new_amenity_list = ["Recreational","Transport & Mobility","Healthcare & Safety", "Technological","Mechanical","Accessibility"]
og_amenity_list = ["Recreational (parks, sport facilities, hiking trails, public beaches, etc)",
                   "Transport & mobility (bus stops, EV charging stations, parking, bicycle lanes, etc)",
                   "Healthcare & Safety (emergency services, hospitals, pharmacies, public defibrillators, etc)",
                   "Technological (public wi-fi, etc)",
                   "Mechanical (water grid, electric grid, etc)",
                   "Accessibility features (wheelchair ramps, tactile pavement, public toilets, etc)"]

new_tool_list = ["Government database","City software","Navigation app"]
og_tool_list = ["Government database (i.e: data.gov.ie)",
                   "City planning or Zoning software",
                   "Navigation applications (i.e: Google Maps)"]

# use mapping dict to account for multiple answers
amenity_mapping = dict(zip(og_amenity_list, new_amenity_list))
users_B["type_amenity_data_work"] = users_B["type_amenity_data_work"].apply(
    lambda x: ";".join([amenity_mapping.get(item.strip(), item.strip()) for item in x.split(";")])
)
tool_mapping = dict(zip(og_tool_list, new_tool_list))
users_B["type_tool_work"] = users_B["type_tool_work"].apply(
    lambda x: ";".join([tool_mapping.get(item.strip(), item.strip()) for item in x.split(";")])
)

# adding "other" prefix to custom amenity answers
users_B["type_amenity_data_work"] = users_B["type_amenity_data_work"].apply(prefix_other_answers, answer=new_amenity_list)

# adding "other" prefix to custom tool answers
users_B["type_tool_work"] = users_B["type_tool_work"].apply(prefix_other_answers, answer=new_tool_list)

# adding "other" prefix to why satisfaction tool work "other" answers
neg_reason_tool_list = ["Incomplete information","Not user friendly","Slow - not modern", "Empty","Unapplicable"]
users_B["why_unsatisfied_tool_work"] = users_B["why_unsatisfied_tool_work"].apply(prefix_other_answers, answer=neg_reason_tool_list)

# adding "other" prefix to additional amenity "other" answers
users_B["other_amenity_work"] = users_B["other_amenity_work"].apply(prefix_other_answers, answer=other_amenity_list)

# adding "other" prefix to why impractical demo "other" answers
users_B["why_impractical_demo_work"] = users_B["why_impractical_demo_work"].apply(prefix_other_answers, answer=neg_reason_demo_list)

In [ ]:
users_B.columns

In [ ]:
# explode mutiple answers cols
userBcols_to_explode = ["device_work","type_amenity_data_work","type_tool_work","satisfaction_tool_work","why_impractical_demo_work","other_amenity_work"]

users_B_expl = explode_multiple_answers(users_B,userBcols_to_explode)

In [ ]:
users_B_expl.head(10)

#### Export to csv user_b

In [ ]:
users_B_expl.to_csv("userB_responses.csv", index=False)